In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp

params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   

    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))


In [ ]:
### read in opto stim data for different frequencies
id_date = { 
"004106" : ["20230908"],
"004155" : ["20230908"],
"004107" : ["20230908"],
"004105" : ["20230728"],
"004103" : ["20230728"],
"004104" : ["20230728"]
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        data = mat_data["response"]["sessdata"]
        laser_times = mat_data["response"]["laser_time"]
        laser_itis = mat_data["response"]["trial_iti"]
        laser_widths = mat_data["response"]["trial_pulse_width"]
        laser_freqs = [np.round((width/(iti + width)) * 100) for iti,width in zip(laser_itis,laser_widths)]
        trans = np.transpose(data)
        ard_times = trans[4]
        wheel = trans[3]
        wheel[0] = 0
        wheel = zscore(np.interp(times,ard_times,wheel))
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(wheel_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "wheel", "ID" : id, "date" : date,"sess":sess,"frequency":str(int(laser_freqs[j]))}
                df_list.append(new_dict)

        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"frequency":str(int(laser_freqs[j]))}
                df_list.append(new_dict)

        for j,trial in enumerate(face_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(face_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "face", "ID" : id, "date" : date,"sess":sess,"frequency":str(int(laser_freqs[j]))}
                df_list.append(new_dict)

freq_df = pd.DataFrame.from_dict(df_list)

In [ ]:
freq_df

In [ ]:
###mcherry control data

id_date = { 
"000033" : [["20221018",2],["20221018",3]],
"000034" : [["20221014",2],["20221019",3]],
"000035" : [["20221014",2],["20221014",3]],
"000036" : [["20221014",2],["20221019",3]],

}

pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date,sess in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        laser_times = loadmat(matpath)["data"]["response"]["laser_time"]
        df = pd.read_csv(pupil_path)
        time = df['Time'].to_numpy()
        areas = df['Pupil Area'].to_numpy()
        trial_matrix,peri_times = trial_align(laser_times,time,areas,pre = pre, post = post)

        for j,trial in enumerate(trial_matrix):
            for i,time in enumerate(peri_times):
                baseline = np.nanmean(trial[160:200])
                trial = trial - baseline
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"ID":id,"date":date,"sess":sess}
                df_list.append(new_dict)

mcherry_df = pd.DataFrame.from_dict(df_list)


In [ ]:
###get new mcherry data
id_date = { 
 "004259" : ["20240513"],
 "004260" : ["20240513"],
"004261" : ["20240513"],
"004262" : ["20240513"],
"004264" : ["20240513"],
}
sess =1
pre = 10
post = 20
df_list = []
labels = list(range(1,5))
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in pupil_snips:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        binned_slope = np.array(pd.qcut(slopes,q = 4,labels = labels))
        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i], "ID" : id, "date" : date,"sess":sess}
                df_list.append(new_dict)

mcherry_df1 = pd.DataFrame.from_dict(df_list)

In [ ]:
mcherry_df = pd.concat([mcherry_df,mcherry_df1])

In [ ]:
plt.figure(figsize = (2.5,2.2))

ax = sns.lineplot(data = freq_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID","type","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1)
sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "grey",linewidth = 1,alpha = 0.5)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,10],[-0.5,4],2,0.5)
ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Pupil Size (z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### freq bar df
id_date = { 
"004106" : ["20230908"],
"004155" : ["20230908"],
"004107" : ["20230908"],
"004105" : ["20230728"],
"004103" : ["20230728"],
"004104" : ["20230728"]
}

sess =1
pre = 10
post = 10
df_list = []
delta_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        data = mat_data["response"]["sessdata"]
        laser_times = mat_data["response"]["laser_time"]
        laser_itis = mat_data["response"]["trial_iti"]
        laser_widths = mat_data["response"]["trial_pulse_width"]
        laser_freqs = [np.round((width/(iti + width)) * 100) for iti,width in zip(laser_itis,laser_widths)]
        trans = np.transpose(data)
        ard_times = trans[4]
        wheel = trans[3]
        wheel[0] = 0
        wheel = zscore(np.interp(times,ard_times,wheel))
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            delta = post_laser - pre_laser
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'type':"wheel","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'type':"wheel","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
            delta_dict = {"Delta" : delta,"ID" : id,'type':"wheel","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            delta_list.append(delta_dict)

    

        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            delta = post_laser - pre_laser
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'type':"pupil","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'type':"pupil","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
            delta_dict = {"Delta" : delta,"ID" : id,'type':"pupil","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            delta_list.append(delta_dict)


        for j,trial in enumerate(face_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[200:220])
            delta = post_laser - pre_laser
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'type':"face","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'type':"face","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
            delta_dict = {"Delta" : delta,"ID" : id,'type':"face","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            delta_list.append(delta_dict)



freq_bar_df = pd.DataFrame.from_dict(df_list)
delta_df = pd.DataFrame.from_dict(delta_list)

In [ ]:
plt.figure(figsize = (1.2,0.9))
g = sns.barplot(data = freq_bar_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue",order = ["Pre-Laser","Post-Laser"])
sns.lineplot(data = freq_bar_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(-1,4,1))
g.set(ylim=(-1, 3))
g.set_ylabel("Pupil Size\n(z-scr)")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_pupil_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = freq_bar_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID",'laser']).mean().reset_index()
group1 = stat_df[stat_df['laser']=="Pre-Laser"]
group2 = stat_df[stat_df['laser']=="Post-Laser"]
wilcoxon(group1["value"], group2["value"],nan_policy= "omit")

In [ ]:
###mcherry bar
###mcherry control data

id_date = { 
"000033" : [["20221018",2],["20221018",3]],
"000034" : [["20221014",2],["20221019",3]],
"000035" : [["20221014",2],["20221014",3]],
"000036" : [["20221014",2],["20221019",3]],

}

pre = 10
post = 10
df_list = []
delta_list = []
for id,dates in id_date.items():
    for date,sess in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        laser_times = loadmat(matpath)["data"]["response"]["laser_time"]
        df = pd.read_csv(pupil_path)
        time = df['Time'].to_numpy()
        areas = df['Pupil Area'].to_numpy()
        trial_matrix,peri_times = trial_align(laser_times,time,areas,pre = pre, post = post)

        for j,trial in enumerate(trial_matrix):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            delta = post_laser - pre_laser


            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'Condition':"mCherry","Date":date,"trial":j}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'Condition':"mCherry","Date":date,"trial":j}
            df_list.append(new_dict)
            delta_dict = {"Delta" : delta,"ID" : id,'Condition':"mCherry","Date":date,"trial":j}
            delta_list.append(delta_dict)

            

bar_df = pd.DataFrame.from_dict(df_list)
mcherry_delta_df = pd.DataFrame.from_dict(delta_list)


In [ ]:
###get new mcherry bar

id_date = { 
"004259": ["20240513"],
"004260": ["20240513"],
 "004261": ["20240513"],
 "004262": ["20240513"],
"004263": ["20240513"],
"004264": ["20240513"],

}
sess =1
pre = 10
post = 10
df_list = []
delta_list = []

for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

           
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'Condition':"mCherry","Date":date,"trial":j}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'Condition':"mCherry","Date":date,"trial":j}
            df_list.append(new_dict)

            new_dict = {"Delta" : post_laser - pre_laser,"ID" : id,'Condition':"mCherry","Date":date,"trial":j}
            delta_list.append(new_dict)





bar_df = pd.DataFrame.from_dict(df_list)
mcherry_delta_df1 = pd.DataFrame.from_dict(delta_list)

In [ ]:
mcherry_delta_df = pd.concat([mcherry_delta_df,mcherry_delta_df1])

In [ ]:
###stats
stat_df = mcherry_delta_df.groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0,0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
plt.figure(figsize = (1.2,0.9))
g = sns.barplot(data = bar_df.groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",errorbar = "se",fill = None,errwidth= 1,edgecolor = "black",order = ["Pre-Laser","Post-Laser"])
sns.lineplot(data = bar_df.groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(-1,4,1))
g.set(ylim=(-1, 3))
g.set_ylabel("Pupil Size\n(z-scr)")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/optomcherry_pupil_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = bar_df.groupby(["ID",'laser']).mean().reset_index()
group1 = stat_df[stat_df['laser']=="Pre-Laser"]
group2 = stat_df[stat_df['laser']=="Post-Laser"]
wilcoxon(group1["value"], group2["value"],nan_policy= "omit")

In [ ]:
###wheel trace
plt.figure(figsize = (1.2,0.9))

ax = sns.lineplot(data = freq_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID","type","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "darkorange",linewidth = 1)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,10],[-0.5,2.5],2,0.5)
ax.xaxis.label.set_visible(False)
ax.set_ylabel("Running Speed\n(z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_wheel.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###face trace
plt.figure(figsize = (1.2,0.9))

ax = sns.lineplot(data = freq_df.query("frequency == '20'").query("type == 'face'").groupby(["ID","type","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "purple",linewidth = 1)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,10],[-0.5,1.5],2,0.5)
ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Facial Movement\n(z-scr)")
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_face.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###wheel bar 
plt.figure(figsize = (1,0.9))
g = sns.barplot(data = freq_bar_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",errorbar = "se",fill = None,errwidth= 1,edgecolor = "darkorange",order = ["Pre-Laser","Post-Laser"])
sns.lineplot(data = freq_bar_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(-1,2.5,1))
g.set(ylim=(-1, 2))
g.set_ylabel("Running Speed\n(z-scr)")
g.xaxis.label.set_visible(False)
g.yaxis.label.set_visible(False)


#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_wheel_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = freq_bar_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID",'laser']).mean().reset_index()
group1 = stat_df[stat_df['laser']=="Pre-Laser"]
group2 = stat_df[stat_df['laser']=="Post-Laser"]
wilcoxon(group1["value"], group2["value"],nan_policy= "omit")

In [ ]:
###face bar 
plt.figure(figsize = (1.2,0.9))
g = sns.barplot(data = freq_bar_df.query("frequency == '20'").query("type == 'face'").groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple",order = ["Pre-Laser","Post-Laser"])
sns.lineplot(data = freq_bar_df.query("frequency == '20'").query("type == 'face'").groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(-1,1,0.5))
g.set(ylim=(-0.5, 0.5))
g.set_ylabel("Facial Movement\n(z-scr)")
g.xaxis.label.set_visible(False)
g.yaxis.label.set_visible(False)


#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_face_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
delta_df

In [ ]:
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = delta_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.stripplot(data = delta_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.25)

g.set_yticks(np.arange(0,5,1))
g.set(ylim=(0, 4))
g.set_ylabel("∆ Pupil Size\n(z-scr)")

g.set_xlabel("ChR2")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = delta_df.query("frequency == '20'").query("type == 'pupil'").groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = mcherry_delta_df.groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "black")
sns.stripplot(data = mcherry_delta_df.groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.25)

g.set_yticks(np.arange(-0.5,2,0.5))
g.set(ylim=(-0.5, 1.5))
g.set_ylabel("∆ Pupil Size\n(z-scr)")
g.set_xlabel("mCherry")


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/mcherry_optodelta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = mcherry_delta_df.groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = delta_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "darkorange")
sns.stripplot(data = delta_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.25)

g.set_yticks(np.arange(0,3,1))
g.set(ylim=(0, 2))
g.set_ylabel("∆ Running\n Speed(z-scr)")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_wheel_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = delta_df.query("frequency == '20'").query("type == 'wheel'").groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = delta_df.query("frequency == '20'").query("type == 'face'").groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple")
sns.stripplot(data = delta_df.query("frequency == '20'").query("type == 'face'").groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.25)

g.set_yticks(np.arange(0,3,1))
g.set(ylim=(0, 2))
g.set_ylabel("Facial Movement\n(z-scr)")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_face_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = delta_df.query("frequency == '20'").query("type == 'face'").groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
###stats
stat_df = freq_bar_df.query("frequency == '20'").query("type == 'face'").groupby(["ID",'laser']).mean().reset_index()
group1 = stat_df[stat_df['laser']=="Pre-Laser"]
group2 = stat_df[stat_df['laser']=="Post-Laser"]
wilcoxon(group1["value"], group2["value"],nan_policy= "omit")

In [ ]:
###different frequency pupil trace
plt.figure(figsize = (2.5,2.2))

ax = sns.lineplot(data = freq_df.query("type == 'pupil'").groupby(["ID","type","Time from Laser (s)","frequency"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,hue = "frequency",legend = None,palette = ["mediumblue","midnightblue","skyblue","dodgerblue"])
format_ax(ax,[-2,10],[-0.5,4],2,0.5)
ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Pupil Size (z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_pupil_diff_freq.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### freq bar delta df
id_date = { 
"004106" : ["20230908"],
"004155" : ["20230908"],
"004107" : ["20230908"],
"004105" : ["20230728"],
"004103" : ["20230728"],
"004104" : ["20230728"]
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        data = mat_data["response"]["sessdata"]
        laser_times = mat_data["response"]["laser_time"]
        laser_itis = mat_data["response"]["trial_iti"]
        laser_widths = mat_data["response"]["trial_pulse_width"]
        laser_freqs = [np.round((width/(iti + width)) * 100) for iti,width in zip(laser_itis,laser_widths)]
        trans = np.transpose(data)
        ard_times = trans[4]
        wheel = trans[3]
        wheel[0] = 0
        wheel = zscore(np.interp(times,ard_times,wheel))
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'type':"wheel","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
          

        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'type':"pupil","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
      

        for j,trial in enumerate(face_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[200:220])
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'type':"face","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)
      

freq_delta_df = pd.DataFrame.from_dict(df_list)

In [ ]:
plt.figure(figsize = (1.2,2.2))
g = sns.barplot(data = freq_delta_df.query("type == 'pupil'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue",order = ["3","5","10","20"])
sns.lineplot(data = freq_delta_df.query("type == 'pupil'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[4:]:
    y1,y2,y3,y4 = line.get_ydata()
    line.set_ydata([y3,y4,y1,y2])
g.set_yticks(np.arange(0,4,0.5))
g.set(ylim=(0, 3.5))
g.set_ylabel("∆ Pupil Size\n(z-scr)")
g.set_xlabel("Laser Freq. (Hz)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_pupil_freqbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = freq_delta_df.query("type == 'pupil'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '3'")["Delta"]
q2 = freq_delta_df.query("type == 'pupil'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '5'")["Delta"]
q3 = freq_delta_df.query("type == 'pupil'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '10'")["Delta"]
q4 = freq_delta_df.query("type == 'pupil'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '20'")["Delta"]
kruskal(q1,q2,q3,q4)

In [ ]:
plt.figure(figsize = (1.2,2.2))
g = sns.barplot(data = freq_delta_df.query("type == 'wheel'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "darkorange",order = ["3","5","10","20"])
sns.lineplot(data = freq_delta_df.query("type == 'wheel'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[4:]:
    y1,y2,y3,y4 = line.get_ydata()
    line.set_ydata([y3,y4,y1,y2])
g.set_yticks(np.arange(0,4,0.5))
g.set(ylim=(0, 3.5))
g.set_ylabel("∆ Running Speed\n(z-scr)")
g.set_xlabel("Laser Freq. (Hz)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_speed_freqbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = freq_delta_df.query("type == 'wheel'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '3'")["Delta"]
q2 = freq_delta_df.query("type == 'wheel'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '5'")["Delta"]
q3 = freq_delta_df.query("type == 'wheel'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '10'")["Delta"]
q4 = freq_delta_df.query("type == 'wheel'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '20'")["Delta"]
kruskal(q1,q2,q3,q4)

In [ ]:
plt.figure(figsize = (1.2,2.2))
g = sns.barplot(data = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple",order = ["3","5","10","20"])
sns.lineplot(data = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[4:]:
    y1,y2,y3,y4 = line.get_ydata()
    line.set_ydata([y3,y4,y1,y2])
g.set_yticks(np.arange(0,4,0.5))
g.set(ylim=(0, 3.5))
g.set_ylabel("∆ Facial Movement\n(z-scr)")
g.set_xlabel("Laser Freq. (Hz)")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_face_freqbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '3'")["Delta"]
q2 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '5'")["Delta"]
q3 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '10'")["Delta"]
q4 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '20'")["Delta"]
kruskal(q1,q2,q3,q4)

In [ ]:
g = sns.barplot(data = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple")
sns.lineplot(data = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
# for line in g.get_lines()[4:]:
#     y1,y2,y3,y4 = line.get_ydata()
#     line.set_ydata([y3,y4,y1,y2])

In [ ]:
###stats
q1 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '3'")["Delta"]
q2 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '5'")["Delta"]
q3 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '10'")["Delta"]
q4 = freq_delta_df.query("type == 'face'").groupby(["ID",'frequency']).mean().reset_index().query("frequency == '20'")["Delta"]
kruskal(q1,q2,q3,q4)

In [ ]:
### read in opto stim data for different durations
id_date = { 
"004103" : ["20230801"],
"004104" : ["20230801"],
"004105" : ["20230801"],
"004155" : ["20230913"],
"004106" : ["20230913"],
"004107" : ["20230913"]
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        data = mat_data["response"]["sessdata"]
        laser_times = mat_data["response"]["laser_time"]
        laser_durs = mat_data["response"]["trial_dur"]
        trans = np.transpose(data)
        ard_times = trans[4]
        wheel = trans[3]
        wheel[0] = 0
        wheel = zscore(np.interp(times,ard_times,wheel))
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(wheel_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "wheel", "ID" : id, "date" : date,"sess":sess,"duration":str(int(laser_durs[j])/1000)}
                df_list.append(new_dict)

        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"duration":str(int(laser_durs[j])/1000)}
                df_list.append(new_dict)

        for j,trial in enumerate(face_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(face_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "face", "ID" : id, "date" : date,"sess":sess,"duration":str(int(laser_durs[j])/1000)}
                df_list.append(new_dict)

dur_df = pd.DataFrame.from_dict(df_list)

In [ ]:
dur_df["duration"].unique()

In [ ]:
dur_df["duration"].value_counts()

In [ ]:
###different frequency pupil trace
plt.figure(figsize = (2.5,2.2))

ax = sns.lineplot(data = dur_df.query("type == 'pupil'").query("duration != '0.05'",engine = "python").groupby(["ID","type","Time from Laser (s)","duration"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,hue = "duration",legend = None,palette = ["dodgerblue","mediumblue","midnightblue"])
format_ax(ax,[-2,10],[-0.5,4],2,0.5)
ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Pupil Size (z-scr)")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_pupil_diff_dur.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
### dur bar delta df
id_date = { 
"004103" : ["20230801"],
"004104" : ["20230801"],
"004105" : ["20230801"],
"004155" : ["20230913"],
"004106" : ["20230913"],
"004107" : ["20230913"]
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        data = mat_data["response"]["sessdata"]
        laser_times = mat_data["response"]["laser_time"]
        laser_durs = mat_data["response"]["trial_dur"]
        trans = np.transpose(data)
        ard_times = trans[4]
        wheel = trans[3]
        wheel[0] = 0
        wheel = zscore(np.interp(times,ard_times,wheel))
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            pre_laser = np.nanmean(trial[160:200])
            duration_frames = int((laser_durs[j]/1000) *20)
            post_laser = np.nanmean(trial[200 + (duration_frames-5):200 + duration_frames])
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'type':"wheel","Date":date,"trial":j,"duration":str(int(laser_durs[j])/1000)}
            df_list.append(new_dict)
          

        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            duration_frames = int((laser_durs[j]/1000) *20)
            post_laser = np.nanmean(trial[200 + (duration_frames-5):200 + duration_frames])
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'type':"pupil","Date":date,"trial":j,"duration":str(int(laser_durs[j])/1000)}
            df_list.append(new_dict)
      

        for j,trial in enumerate(face_snips):
            pre_laser = np.nanmean(trial[160:200])
            duration_frames = int((laser_durs[j]/1000) *20)
            post_laser = np.nanmean(trial[200 + (duration_frames-5):200 + duration_frames])

            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'type':"face","Date":date,"trial":j,"duration":str(int(laser_durs[j])/1000)}
            df_list.append(new_dict)
      
dur_delta_df = pd.DataFrame.from_dict(df_list)

In [ ]:
plt.figure(figsize = (1.2,2.2))
g = sns.barplot(data = dur_delta_df.query("duration != '0.05'",engine = "python").query("type == 'pupil'").groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = dur_delta_df.query("duration != '0.05'",engine = "python").query("type == 'pupil'").groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(-1,6,1))
g.set(ylim=(-1, 5))
g.set_ylabel("∆ Pupil Size\n(z-scr)")
g.set_xlabel("Laser Duration (s)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_pupil_durbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = dur_delta_df.query("type == 'pupil'").groupby(["ID",'duration']).mean().reset_index().query("duration == '0.25'")["Delta"]
q2 = dur_delta_df.query("type == 'pupil'").groupby(["ID",'duration']).mean().reset_index().query("duration == '1.0'")["Delta"]
q3 = dur_delta_df.query("type == 'pupil'").groupby(["ID",'duration']).mean().reset_index().query("duration == '3.0'")["Delta"]
kruskal(q1,q2,q3)

In [ ]:
plt.figure(figsize = (1.2,2.2))
g = sns.barplot(data = dur_delta_df.query("duration != '0.05'",engine = "python").query("type == 'wheel'").groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "darkorange")
sns.lineplot(data = dur_delta_df.query("duration != '0.05'",engine = "python").query("type == 'wheel'").groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(-1,6,1))
g.set(ylim=(-1, 5))
g.set_ylabel("∆ Running Speed\n(z-scr)")
g.set_xlabel("Laser Duration (s)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_wheel_durbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = dur_delta_df.query("type == 'wheel'").groupby(["ID",'duration']).mean().reset_index().query("duration == '0.25'")["Delta"]
q2 = dur_delta_df.query("type == 'wheel'").groupby(["ID",'duration']).mean().reset_index().query("duration == '1.0'")["Delta"]
q3 = dur_delta_df.query("type == 'wheel'").groupby(["ID",'duration']).mean().reset_index().query("duration == '3.0'")["Delta"]
kruskal(q1,q2,q3)

In [ ]:
plt.figure(figsize = (1.2,2.2))
g = sns.barplot(data = dur_delta_df.query("duration != '0.05'",engine = "python").query("type == 'face'").groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple")
sns.lineplot(data = dur_delta_df.query("duration != '0.05'",engine = "python").query("type == 'face'").groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(-1,6,1))
g.set(ylim=(-1, 5))
g.set_ylabel("∆ Facial Movement\n(z-scr)")
g.set_xlabel("Laser Duration (s)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/opto_face_durbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = dur_delta_df.query("type == 'face'").groupby(["ID",'duration']).mean().reset_index().query("duration == '0.25'")["Delta"]
q2 = dur_delta_df.query("type == 'face'").groupby(["ID",'duration']).mean().reset_index().query("duration == '1.0'")["Delta"]
q3 = dur_delta_df.query("type == 'face'").groupby(["ID",'duration']).mean().reset_index().query("duration == '3.0'")["Delta"]
kruskal(q1,q2,q3)